# 0. Installation and import the libraries

FIRST THING FIRST. 

Install sylfenUtils==1.18.1 in a virtual environment.

In order to run your jupyter-notebook with your virtual environment follow instructions [here](https://queirozf.com/entries/jupyter-kernels-how-to-add-change-remove).
Then import the libraries.

In [ ]:
from sylfenUtils import comUtils
import os,sys,re, pandas as pd

Check that you have installed the right version of sylfenUtils.

# 1.Build the configuration of your project using the Conf_generator
For convenience put your configuration work in a conf.py file which should serve as an object with all
the important information.

Let's say you have a modbus table of a device in a "dummy_devices.csv" file and a modbus_map  in a "dummy_modbbus_devices.ods". Read it with pandas without forgetting to adjust the path of the file depending where it is located on your device. 

In [ ]:
dummy_df_devices = pd.read_csv('data/dummy_devices.csv', index_col=0).iloc[[0]]
dummy_df_devices

In [ ]:
dummy_modbus_map = pd.read_excel('data/dummy_modbus_devices.ods', sheet_name = 'dummy1', index_col = 0)
dummy_modbus_map

# 2. INSTANCIATE A GAIA OBJECT

In [ ]:
def generate_dummy_conf():
    return {
        'df_devices': dummy_df_devices,
        'modbus_maps' : {'dummy1':dummy_modbus_map}
    }

In [ ]:
from sylfenUtils import GAIA

dummy_gaia=GAIA.GAIA(project_name='dummy_project_gaia', 
                     function_generator=generate_dummy_conf, 
                     root_folder=None, realtime=True)

It has created a folder in the home of the user executing the script with following content : 

- **dummy_project_daily** would contain(by default) the data that are going to be generated => dummy_conf.FOLDERPKL
- **conf_dummy_project.pkl** is the object containing all the important information of the project(the configuration) saved as dictionnary. The keys are then attributes of dummy_conf object.  
- **log** : dummy_conf.LOG_FOLDER which is the folder where the logs are going to be stored (by default)
- **parameters.conf** : is a standard file with default settings that you are able to to modify.  

Your configuration is done now. 

**Note**: You can also create your own class to generate the configuration that inheritates from Conf_generator class.

## 1.2 Modify default settings if you need it

As said in previous section you can change the default settings at any time.
For the moment the content of the **parameters.conf** file is :  

In [ ]:
with open('/home/marc-a/dummy_project_gaia_user/parameters.conf','r') as f : 
    for l in f.readlines():
        print(l)

Explanation of the parameters:
- **database parameters**:For the data acquisition you NEED TO INSTALL POSTGRESSQL and CREATE A DATABASE WITH A USER having permissions to create databases. Follow instructions [here](http://wiki.sylfen.com/sql/)
  - db_host : host of the postgres database
  - db_port : port of the postgres database
  - db_name : name of the database.
  - db_user : valid postgressql user with
  - db_password : db_user password to connect to the database
  - DB_TABLE : database table where the data are going to be stored with timestamp,tag,value columns.
- **other shared variables**:  
  - PARKING_TIME:the size of the database in seconds until it is emptied.
  - TZ_RECORD: the timezone used to park the data. By default it is CET whereas it would be better to have UTC. But for people living in CET timezone it is easier to open a day-folder with all the hours  of the current day(and not having to open 2 folders for that).
  - SIMULATOR:whether the simulator should be used instead of connecting to the real devices(convenient when working offline or not on the private network where the devices are available).
  - FOLDERPKL:path of the folder where the daily parked data are stored. If *default* the data are found in dummy_conf.project_folder.
  - LOG_FOLDER:path of the folder where the logs are found.   
Precise where the data will be stored on your machine, which timezone, how big the database will be and the logs folder.
- **dashboard specific variables**
  - TEST_ENV:if True the *dashboard.py* file will be run on port 15000 from flask(not wsgi production environment).
  - TMAX: works only if TEST_ENV is true. The end datetime of datetimepicker of the the dashboard.  

These parameters are then accessible from any instance of Conf_generator (or your own children class) as soon as with a name of the project. As such all the other higher-level objects will know these information.

# 2. DUMP DATA 

## 2.1 Start the SIMULATOR of the modbus device (optionnal)
As we don't have the dummy device delivering data with the modbus protocole, we will create a simulator of the device based on the same modbus map and we will run the modbus server to serve data (random data). 
For your real project you can of course skip this step if the device is available on your network.

In [ ]:
from sylfenUtils.Simulators import SimulatorModeBus

dummy_simulator = SimulatorModeBus(
   port = dummy_gaia.devices['dummy1'].port,
#    port = 6500,
    modbus_map = dummy_gaia.devices['dummy1'].modbus_map,
    simulator_name = 'dummy_gaia_simulator',
    bo = dummy_gaia.devices['dummy1'].byte_order,
    wo = dummy_gaia.devices['dummy1'].word_order)

dummy_simulator.start()

In [ ]:
# Start dumping data
# param bool park_on_time: if True, it will wait for the parking to start at a round time.
dummy_gaia.start_dumping(park_on_time=False)

In [ ]:
dummy_gaia.read_db()

In [ ]:
dummy_gaia.stop_dumping()

In [ ]:
dummy_gaia.run_GUI()

- to deploy it to the world, open the port of the app on your routeur and use host='0.0.0.0'

As Flask says it is not a reliable as a production environment. It is better to use wsgi and gunicorn to run the app.
- more convenient is to control the process with with **systemctl** on a Linux server. 
- Then use **nginx**(or apache) to make the redirection via a reverse proxy configured in a <myapp>.conf file in the nginx folders *sites_enabled* and *sites-available* to be able to map your domain name or subdomain name to the app. A guide is to be found [here](https://github.com/dorianSylfen/smallpower).